### Reaction Diffusion System - Gray Scott Model

Inspiration for this assignment is drawn from nature and its brilliant designs, particularly the formation of Turing patterns in animals. Turing patterns are distinctive and characterized by stripes, spots and combinations of both. In this assignment, we explore the Gray-Scott model of the reaction diffusion system. We approach this problem using the finite element method (FEM), simulate the results with NGSolve and experiment with different parameters to investigate the system's behaviour.
The Gray Scott Reaction Diffusion Model belongs to the class of parabolic partial differential equations (PDEs) because it is time-dependent.
It consists of the following equations:
$$\begin{aligned}
    \frac{\partial u}{\partial t} &= D_u \nabla^2 u - uv^2 + F(1 - u) \\
    \frac{\partial v}{\partial t} &= D_v \nabla^2 v + uv^2 - (F + k)v
\end{aligned} $$
where:
- $u$ and $v$ are the concentrations of the corresponding chemicals $U$ and $V$, not defined in a specific unit.
- $D_u$, $D_v$ are the diffusion rates for $u$ and $v$ respectively. 
- $F$ is the feed rate, representing the rate $U$ is introduced 
- $k$ is the kill rate, representing the rate $U$ becomes inhibited by $V$
- $uv^2$ represents the reaction term, describing how $U$ and $V$ interact.

Our first step is to create a rectangular mesh containing triangular elements and having periodic boundaries.
This means that opposite boundaries are linked to each other and their elements and nodes are mapped to the ones on the opposite side and have the same values. This causes a warp-around effect allowing seamless transitions. Containing around 3600 elements, our mesh has sufficient size for the aiming simulations. Next, we create the corresponding finite element space. First order polynomials were used for stability since higher-order ones were found to create overflow errors.

### Variational Problem
The follow-up step is to derive the weak form of our equations from the strong form.
To derive the weak form, we multiply each equation by a test function $\phi$ and integrate over our domain $\Omega$. 
For the diffusion term we integrate and use Green's first identity.
Over the periodic boundary, the flux terms at opposite boundaries cancel out, therefore the integral over $\partial \Omega$ is equal to zero.
##### Weak form for  u:
$$
\int_\Omega \frac{\partial u}{\partial t} \phi \, dx
= -\int_\Omega D_u \nabla u \cdot \nabla \phi \, dx + \int_{\partial \Omega} (\nabla u \cdot \mathbf{n}) \phi\,ds
- \int_\Omega (u v^2) \phi \, dx 
+ \int_\Omega F(1 - u) \phi \, dx
$$

$$\int_\Omega \frac{\partial u}{\partial t} \phi \, dx
= -\int_\Omega D_u \nabla u \cdot \nabla \phi \, dx - \int_\Omega (u v^2) \phi \, dx 
+ \int_\Omega F(1 - u) \phi \, dx, 
\qquad \forall \, \phi \in H_0^1(\Omega) \; \forall t \, \in (0,T)
$$
##### Weak form for v:

$$
\int_\Omega \frac{\partial v}{\partial t} \phi \, dx
= -\int_\Omega D_v \nabla v \cdot \nabla \phi \, dx +
\int_{\partial \Omega} (\nabla v \cdot \mathbf{n}) \phi \, ds
+ \int_\Omega (u v^2) \phi \, dx 
- \int_\Omega (F + k)v \phi \, dx $$

$$\int_\Omega \frac{\partial v}{\partial t} \phi \, dx
= -\int_\Omega D_v \nabla v \cdot \nabla \phi \, dx + \int_\Omega (uv^2) \phi \, dx 
- \int_\Omega (F + k)v \phi \, dx, 
\qquad \forall \, \phi \in H_0^1(\Omega) \; \forall t \, \in (0,T) $$

We name the reaction terms as $f_u$ and $f_v$:
$$ f_u = \int_\Omega \phi \left( -u v^2 + F(1 - u) \right) \, dx $$

$$f_v = \int_\Omega \phi \left( u v^2 - (F + k)v \right) \, dx $$

The initial condition is set for $t = 0$:
It is essential to have a small disturbance so that the chemicals $U$ and $V$ start reacting.
We choose the concentration $u$ to be taken from a random sample of the uniform distribution $U(0.8, 1)$
and $v$ to be taken from a random sample of the uniform distribution $U(0, 0.2)$.
The unknown functions $u ,v $ are considered as functions with values in the Hilbert space $H^1$:

We are now able to express the equations as Hilbert-space valued ordinary differential equations:

$$
\frac{d u }{dt }(t) + D_u K u(t) = f_u(t)
$$

$$
\frac{d v }{dt }(t) + D_v K v(t) = f_v(t)
$$
### Spatial discretization
Let $V_h$ be a finite element subspace of $H^1(\Omega)$, with basis functions $\{\phi_1(x), \ldots,\phi_N(x)\}$. 
The functions $u$ and $v$ are time-dependent and we may approximate them by:
$$ u_h(t):[0,T] \rightarrow V_h \\
   v_h(t):[0,T] \rightarrow V_h$$
They may be expressed as: 
$$    u_h(t) = \sum_{i=1}^N u_i(t) \phi_i(x), \quad v_h(t) = \sum_{i=1}^N v_i(t) \phi_i(x),
$$
where $u_i(t)$ and $v_i(t)$ are time-dependent coefficients.
We insert this expansion into the variational formulation, and use spatial test-functions $\phi = \phi_j$ :

$$
\int_\Omega \frac{\partial}{\partial t} \Big\{ \sum_{i=1}^N u_i(t) \phi_i(x) \Big\} \phi_j(x) dx + 
\int_\Omega \nabla \Big\{ \sum_{i=1}^N u_i(t) \phi_i(x) \Big\} \; \nabla \phi_j(x) = \int_\Omega f_u(x,t) \phi_j(x) \, dx
\qquad \forall \, j = 1 \ldots N, \forall \, t \in (0,T)
$$
$$
\int_\Omega \frac{\partial}{\partial t} \Big\{ \sum_{i=1}^N v_i(t) \phi_i(x) \Big\} \phi_j(x) dx + 
\int_\Omega \nabla \Big\{ \sum_{i=1}^N v_i(t) \phi_i(x) \Big\} \; \nabla \phi_j(x) = \int_\Omega f_v(x,t) \phi_j(x) \, dx
\qquad \forall \, j = 1 \ldots N, \forall \, t \in (0,T)
$$

The **mass matrix** $M \in R^{N \times N}$ is:

$$
M = \left\{ \int_\Omega \phi_i \phi_j \, dx \right\}_{i,j = 1, \ldots N}
$$
and the **stiffness matrix** $K \in R^{N \times N}$ is:

$$
K = \left\{ \int_\Omega \nabla \phi_i \nabla \phi_j \, dx \right\}_{i,j = 1, \ldots N}
$$
Finally, we obtain the ordinary differential equation:
$$\begin{aligned} M \frac{du}{dt} + D_u K u &= f_u, \\
M \frac{d v}{dt} + D_v K v &= f_v \end{aligned}$$
### Time discretization
We have opted to use forward (explicit) Euler time scheme. Its inherent instability does not compromise our model, while its much lower computational cost significantly reduces execution time.
$$
M \frac{u^{n+1}-u^{n}}{\Delta t} + D_uKu^{n} = f_u^n \Rightarrow
M \frac{u^{n+1}-u^{n}}{\Delta t}  = f_u^n - D_uKu^n \Rightarrow$$
$$\frac{u^{n+1}-u^{n}}{\Delta t}  = M^{-1} ( f_u^n -D_uKu^n) \Rightarrow
u^{n+1}=u^{n}+\Delta t M^{-1}(f_u^n- D_uKu^n )
$$
We follow the same procedure for $v$:
$$
M \frac{v^{n+1}-v^{n}}{\Delta t} + D_vKu^n = f_v^n \Rightarrow
M \frac{v^{n+1}-v^{n}}{\Delta t}  = f_v^n - D_vKv^n \Rightarrow$$
$$\frac{v^{n+1}-v^{n}}{\Delta t}  = M^{-1}(f_v^n -D_vKv^n ) \Rightarrow
v^{n+1}=v^{n}+\Delta t M^{-1}(f_v^n- D_vKv^n )
$$

In conclusion, we formulated the Gray-Scott reaction-diffusion system as a weak variational problem, discretized it using the finite element method, and implemented an explicit Euler time-stepping scheme.
Our last step is to visualize the emerging Turing Patterns using NGSolve.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
import time
from netgen.geom2d import *

In [ ]:
# Create a square in the geometry
periodic = SplineGeometry()
pnts = [ (0,0), (40,0), (40,40), (0,40) ]
pnums = [periodic.AppendPoint(*p) for p in pnts]

# These will be our master edges so we need to save their number.
topbot = periodic.Append(["line", pnums[0], pnums[1]], bc="periodic")
lright = periodic.Append(["line", pnums[1], pnums[2]], bc="periodic")
# Minion boundaries must be defined in the same direction as master ones.
# This is why the the point numbers of this spline are defined in the reverse direction,
# leftdomain and rightdomain must therefore be switched as well!
# We use the master number as the copy argument to create a minion edge.
periodic.Append(["line", pnums[3], pnums[2]], leftdomain=0, rightdomain=1, copy=topbot, bc="periodic")
periodic.Append(["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, copy=lright, bc="periodic")

# Generate the mesh
mesh = Mesh(periodic.GenerateMesh(maxh=1)) 
Draw(mesh) 

# Define the finite element space over the given mesh
fes = Periodic(H1(mesh, order=1))
print("vertices = ", mesh.nv, "| elements = ", mesh.ne)   # number of vertices & elements

In [3]:
# Define trial and test functions
w = fes.TrialFunction()  
phi_w = fes.TestFunction() 

# Assemble the mass and stiffness matrices
mass = BilinearForm(fes)
mass += SymbolicBFI(w * phi_w)
mass.Assemble()
mass_matrix_inverse = mass.mat.Inverse(freedofs=fes.FreeDofs()) # We will use the inverse during the time step

stiffness = BilinearForm(fes)
stiffness += SymbolicBFI(grad(w) * grad(phi_w))  
stiffness.Assemble()
stiffness_matrix = stiffness.mat

In [4]:
def InitializeUV():
    # GridFunctions for solution
    U = GridFunction(fes)  # u solution
    V = GridFunction(fes)  # v solution
    
    # Initial conditions for u and v
    n = len(U.vec)
    U.vec.data = np.random.uniform(0.8, 1, n)  # random values ranging from 0.8 to 1
    V.vec.data = np.random.uniform(0, 0.2, n)  # random values ranging from 0 to 0.2
    return U, V
  
    
def sagma(D_u, F, k, dt, iterations, plotmin=0.25, plotmax=1, D_v=None):  # Simulated Anisotropic Growth and Morphogenesis Algorithm
    if not D_v:
        D_v = 0.5 * D_u # A good rule of thumb
    
    # Initialization
    U, V = InitializeUV()  # solutions
    U_storage = GridFunction(fes, multidim=0) # multidimensional visualization output
    U_storage.AddMultiDimComponent(U.vec) # store initial state

    # Time step
    for t in range(iterations + 1):
        # Compute the reaction terms
        f_u = LinearForm(fes)
        f_u += (-U * V * V + F * (1 - U)) * phi_w * dx
        f_u.Assemble()

        f_v = LinearForm(fes)
        f_v += (U * V * V - (F + k) * V) * phi_w * dx
        f_v.Assemble()

        # Solve for the next time step
        U.vec.data += dt * (mass_matrix_inverse * (f_u.vec - D_u * stiffness_matrix * U.vec))
        V.vec.data += dt * (mass_matrix_inverse * (f_v.vec - D_v * stiffness_matrix * V.vec))
        
        # Visualization 
        steplength = int(iterations/100) # fixed to draw 100 samples
        if (t % steplength == 0):
            print(f"Computing ... {round(100*t/iterations)}% complete.", end="\r")
            U_storage.AddMultiDimComponent(U.vec) # store current state
    
    # Interpolate between the 100 samples for animation smoothness
    print("Interpolating... please be patient.") 
    Draw(U_storage, interpolate_multidim=True, min = plotmin, max = plotmax, autoscale = False)
    Draw(V)


In [5]:
# Example initial state
U, V = InitializeUV()
Draw(U)
Draw(V)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [ ]:
sagma(D_u = 0.0164, F = 0.050, k = 0.061, dt = 2, iterations = 2200)  # untitled 7

In [ ]:
sagma(D_u=0.0125, F=0.035, k=0.0615, dt=3, iterations = 3500)  # amoeba

In [ ]:
sagma(D_u = 0.016, F = 0.050, k = 0.0605, dt = 2, iterations = 1500) # not lines but not dots

In [ ]:
sagma(D_u=0.013, F=0.045, k=0.0595, dt=1, iterations = 2000) # leopard-ly

In [ ]:
# For further experimentation!

#sagma(D_u = 0.016, F = 0.050, k = 0.060, dt = 1, iterations = 3000)  # dots
#sagma(D_u = 0.016, F = 0.050, k = 0.061, dt = 2, iterations = 2000)  # very good lines-corals
#sagma(D_u=0.0165, F=0.048, k=0.0605, dt=1.5, iterations = 1500)  # leopard-ly same but nice
#sagma(D_u=0.064, F=0.035, k=0.057, dt=1, iterations = 1500) # big leopard before overflow
#sagma(D_u = 0.0164, F = 0.050, k = 0.06, dt = 1, iterations = 2000)  # untitled 7 dots version
#sagma(D_u = 0.011, F = 0.041, k = 0.061, dt = 2, iterations = 2000) #swirly and dotty
